In [ ]:
import numpy as np
from scipy.integrate import cumulative_trapezoid
from scipy.signal import find_peaks
from scipy import signal
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from ipywidgets import widgets
from ipyfilechooser import FileChooser

todo: don't use column names, only indices and skip rows that are all Nan or string

In [ ]:
df = pd.read_excel('./data/CPET ambu anonyme/153620_28_4_2021_15_7excelrawdata.xlsx', skiprows=2)
# df = pd.read_excel('./data/CPET ambu anonyme/318092_29_1_2021_14_57excelrawdata.xlsx', header=None)
# (~df.isna()).any(axis=1).idxmax()
df

In [ ]:
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

In [ ]:
@dataclass
class TestFile:
    excel_path: Path
    raw_path: Path
    label: str

def list_test_file_pairs(data_dir: Path) -> list[TestFile]:
    # Todo update this logic for files with name as prefix
    files: list[TestFile] = []
    for xls in data_dir.glob('*.xlsx'):
        prefix = xls.name.split('_')[0]
        log_match = list(data_dir.glob(f'{prefix}*raw.log'))
        if len(log_match) == 0:
            print(f'No matching raw log for Excel file {xls}')
        elif len(log_match) > 1:
            print(f'More then 1 matching raw log for Excel file {xls}?')
        else:
            files.append(TestFile(xls, log_match[0], label=prefix))
    return files

In [ ]:
def load_dataframes(file: TestFile):
    # some xls files have empty rows at the top, so read once to know how many to skip
    df_cycles = pd.read_excel(file.excel_path, header=None)
    n_empty_rows = (~df.isna()).any(axis=1).idxmax()
    df_cycles = pd.read_excel(file.excel_path, skiprows=n_empty_rows)
    assert df_cycles['Unnamed: 0'][1] == 'Unité'
    assert df_cycles['Unnamed: 0'][2] == 'Théor.'
    # df_cycles = df_cycles.drop('Unnamed: 0', axis=1)
    df_cycles = df_cycles.drop('Temps', axis=1)
    df_cycles = df_cycles.drop([0, 1, 2])
    keep_columns = {'Unnamed: 0', 'Vol.Cour.', 'VIn', 'tIn', 'VEx', 'tEx', 'ttot'}
    df_cycles = df_cycles.drop([col for col in df_cycles.columns if col not in keep_columns], axis=1)
    df_cycles = df_cycles.rename(columns={
        'Unnamed: 0': 'phase',
        'Vol.Cour.': 'vol_instant',
        'VIn': 'vol_in',
        'VEx': 'vol_ex',
        'tIn': 't_in',
        'tEx': 't_ex',
        'ttot': 'duration'
    })
    df_cycles['phase'] = df_cycles['phase'].replace('Repos', 'rest')
    df_cycles['phase'] = df_cycles['phase'].replace('Charge', 'load')
    df_cycles['phase'] = df_cycles['phase'].replace('Récupération', 'recovery')
    df_cycles['phase'] = df_cycles['phase'].ffill()

    # drop rows with missing/'-' values at the end
    df_cycles = df_cycles.drop(df_cycles[df_cycles['vol_ex'] == '-'].index)
    float_cols = ['vol_instant', 'vol_in', 't_in', 'vol_ex', 't_ex', 'duration']
    for col in float_cols:
        df_cycles[col] = df_cycles[col].astype(float)
    df_cycles = df_cycles.reset_index(drop=True)

    df_raw = pd.read_csv(file.raw_path, delimiter='\t', names=['t', 'flow', 'fo2', 'fco2'])
    df_raw.set_index('t')
    return df_cycles, df_raw

def analyze(file: TestFile):
    df_raw, df_cycles = load_dataframes(file)
    
    sampling_freq_hz = 1 / (df_raw['t'][:-1] - df_raw['t'].shift(1)[1:]).mean()
    filter_freq_hz = 2*1e-6
    df_raw['instant_vol_raw'] = cumulative_trapezoid(y=df_raw['flow'], x=df_raw['t'], initial=0) 
    sos = signal.butter(4, Wn=filter_freq_hz * sampling_freq_hz, btype='highpass', output='sos')
    flow_filtered = signal.sosfilt(sos, df_raw['flow'])
    df_raw['instant_vol'] = cumulative_trapezoid(y=flow_filtered, x=df_raw['t'], initial=0) 
    print(f'Sum over all instantaneous volume: {df_raw.instant_vol.sum()}')

    peaks, peakprops  = signal.find_peaks(df_raw['instant_vol'], prominence=0.12)
    valls, vallprops = signal.find_peaks(-df_raw['instant_vol'], prominence=0.12)
    print(f'Found {len(peaks)} peaks, {len(valls)} valleys')
    # index of first peak that comes after first valley
    first_peak_idx = np.argwhere(peaks > valls[0]).min()
    # index of last peak that comes before last valley
    last_peak_idx = np.argwhere(valls[-1] < peaks).min()
    first_peak_idx, last_peak_idx
    print(f'Dropping first {first_peak_idx} and last {len(peaks) - last_peak_idx} peaks')
    peaks = peaks[first_peak_idx:last_peak_idx]

    #   p   p   p   n peaks
    #  / \_/ \_/ \
    # v   v   v   v n+1 valleys

    # That makes n complete cycles

    assert len(peaks) == len(valls) - 1
    iv = df_raw['instant_vol']
    vins = iv[peaks].array - iv[valls[:-1]].array
    vexs = iv[peaks].array - iv[valls[1:]].array
    print(f'{len(peaks)} cycles')

    pad_cycle_data = len(vins) - len(df_cycles['vol_in'])
    cyc_vol_in_padded = np.pad(df_cycles['vol_in'].array, (pad_cycle_data, 0))
    corr = np.correlate(cyc_vol_in_padded, vins, 'full')
    shift = corr.argmax() - (len(vins) - 1)
    drop_highres_cycles_front = pad_cycle_data - shift
    drop_highres_cycles_back = shift 
    vins_trimmed = vins[drop_highres_cycles_front:len(vins) - drop_highres_cycles_back]
    peaks_trimmed = peaks[drop_highres_cycles_front:len(peaks) - drop_highres_cycles_back]
    valls_trimmed = valls[drop_highres_cycles_front:len(valls) - drop_highres_cycles_back]
    peaks_trimmed.shape, vins.shape, vins_trimmed.shape, valls_trimmed.shape
    df_cycles['highres_t_start'] = df_raw['t'][valls_trimmed[:-1]].values
    df_cycles['highres_t_max'] = df_raw['t'][peaks_trimmed].values
    df_cycles['highres_t_end'] = df_raw['t'][valls_trimmed[1:]].values
    df_cycles['highres_duration'] = df_cycles['highres_t_end'] - df_cycles['highres_t_start']
    return df_cycles, df_raw

In [ ]:
def on_analyze_clicked(file: TestFile):
    global df_cycles
    global df_raw
    df_cycles, df_raw = analyze(file)

def on_dir_chosen(chooser):
    data_dir = Path(chooser.value)
    files = list_test_file_pairs(data_dir)
    select = widgets.Select(options=[(file.label, file) for file in files], layout={'height': '300px'})
    display(select)
    button = widgets.Button(description='Analyze')
    button.on_click(lambda button: on_analyze_clicked(select.value))
    display(button)


In [ ]:
fc = FileChooser()
fc.show_only_dirs=True
fc.default_path = '/home/thomas/p/soupirs/data/CPET ambu anonyme'
fc.title = 'Pick folder with raw csv and excel files'
fc.register_callback(on_dir_chosen)
fc

*Edge cases*
|file|time|
|-|-|
|ambu 1091879|836

In [ ]:
df_cycles.loc[(df_cycles['highres_duration'] - df_cycles['duration']).abs() > 0.1]

In [ ]:
px.line(df_cycles, y=['duration', df_cycles['highres_t_end'] - df_cycles['highres_t_start']], hover_data=['highres_t_start'])

In [ ]:
df_raw['vol_cycle_start'] = pd.Series(dtype=float)
df_raw.loc[df_raw['t'].isin(df_cycles['highres_t_start'].values), 'vol_cycle_start'] = df_raw[df_raw['t'].isin(df_cycles['highres_t_start'].values)]['instant_vol']
fig = px.line(df_raw, x='t', y='instant_vol')
fig = fig.add_scatter(x=df_raw.t, y=df_raw['vol_cycle_start'], mode='lines+markers')
fig.show()

In [ ]:
df_correlated = pd.DataFrame()
df_correlated['vol_in_highres'] = vins_trimmed
df_correlated['vol_in_cycles'] = df_cycles['vol_in']
px.scatter(df_correlated, df_correlated.index, ['vol_in_highres', 'vol_in_cycles'], title='Vol Insp cycles/highres')

In [ ]:
vexs_trimmed = vexs[drop_highres_cycles_front:len(vexs)-drop_highres_cycles_back]
cyc_vol_ex = df_cycles['vol_ex']
df_correlated['vol_ex_highres'] = vexs_trimmed
df_correlated['vol_ex_cycles'] = df_cycles['vol_ex']
px.scatter(df_correlated, df_correlated.index, ['vol_ex_highres', 'vol_ex_cycles'], title='Vol Exp cycles/highres')

In [ ]:
px.line(df_raw, x='t', y=['instant_vol_raw', 'instant_vol', 'flow'])
# px.scatter(df_raw, x='t', y=peaks)
# px.line(df_raw, x='t', y=['flow', 'flow_ma', 'filtered', 'instant_vol'])